In [ ]:
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
import re
import math
import numpy as np
from numpy import dot
from numpy.linalg import norm
from scipy import spatial
import matplotlib.pyplot as plt
import pickle

 #nltk.download('stopwords')
 #nltk.download('punkt')

In [ ]:
def snow_ball_stemmer(data_list):
  stem = SnowballStemmer(language='english')
  return list(map(lambda word:stem.stem(word),data_list))

In [ ]:
def text_cleaner(data_str):
  data_str = data_str.strip(' ').lower()
  data_str = data_str
  new_str= ''
  for char in data_str:
    if char not in '''!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~''':
      new_str +=char
  return new_str.strip(' ')

In [ ]:

# Takes a list of words as parameter and returns a new list after removing the stopwords
def remove_stop_words(data_list):
  new_list = []
  for word in data_list:
    if word not in nltk.corpus.stopwords.words('english'):
      new_list.append(word)
  return new_list


In [ ]:
def preprocess(data):
  return remove_stop_words(nltk.word_tokenize(text_cleaner(data)))

In [ ]:
inverted_index = {}
# cat 3
# 'cat' = [1,2,5]

# Takes a term and its corresponding documentID and adds to the inverted index
def add_term_to_inverted_index(term,documentID):
  # an inverted index is a dict contained term,globalCount to posting list and each posting list contains ()
  try:
    # print(inverted_index[term]['count'])
    # inverted_index[term]['count']+=1 # increment the term count
    
    for document in inverted_index[term]['posting_list'].copy():
      if document['docID']>documentID:
      # if documentID in [i['docID'] for i in inverted_index[term]['posting_list']]:
        # we don't have that particular document in the posting list, add it
        inverted_index[term]['posting_list'].append({'docID':documentID,'count':1})
        inverted_index[term]['count']=len(inverted_index[term]['posting_list'])
        inverted_index[term]['posting_list'] = sorted(inverted_index[term]['posting_list'],key=lambda x:x['docID'])
        break
      elif document['docID']==documentID:
        document['count']+=1
        break
    
    else:
      inverted_index[term]['posting_list'].append({'docID':documentID,'count':1})
      inverted_index[term]['count']=len(inverted_index[term]['posting_list'])
      inverted_index[term]['posting_list'] = sorted(inverted_index[term]['posting_list'],key=lambda x:x['docID'])



  except KeyError:
    # print('we failed')
    # We are adding a term to our index keep the count empty and an empty list for the posting_list
    inverted_index[term]={'count':1,'posting_list':[{'docID':documentID,'count':1}]}
    # print(inverted_index)



In [ ]:
# for temp1 in inverted_index.copy().keys():
#   inverted_index[temp1]['count']=len(inverted_index[temp1]['posting_list'])

In [ ]:
# Convert all unknown attributes to NaN
dataframe_original = pd.read_csv(r"https://drive.google.com/file/d/1SigikUJF9OCigB7juNrx8R8EkKX_Cx0l/view?usp=sharing")

dataframe = dataframe_original.replace(['Unknown','unknown'],'')
# dataframe.drop('Wiki Page',1,inplace=True)
dataframe.drop(columns="Wiki Page",inplace=True)


In [ ]:
# debug head(10)
total_doc=5000
retried_doc=30

for temp in dataframe.head(total_doc).iterrows():
  value = ''

  docID = temp[0]
 
  for temp_value in temp[1].to_dict().values():
    value+=str(temp_value)+' ' 

  value = value.replace('nan','')
  processed_list = preprocess(value)

  # change to a lemmatizer instead
  # print(snow_ball_stemmer(processed_list))
  # print(processed_list)
  for term in processed_list:
    add_term_to_inverted_index(term,docID)

  

In [ ]:
# inverted_index

In [ ]:
# from pprint import pprint
# pprint(inverted_index)

In [ ]:
def idf_func(c):
  
  return math.log2(total_doc/c)


In [ ]:
def calc_idf():
  for term in inverted_index.keys():
    c=inverted_index[term]['count']
    idf_value=idf_func(c)
    inverted_index[term]['idf']=idf_value
calc_idf()


#CALCULATING TF-IDF SCORE

In [ ]:
tfidf_dict={}
tfidf_list=[]

for term in inverted_index.keys():
  tfidf_list.append(term)
  temp=inverted_index[term]['posting_list']
  for t in temp:
    tfidf_dict[t['docID'],term]=inverted_index[term]['idf']*(1+math.log2(t['count']))
    

# print(tfidf_dict)

In [ ]:
# from pprint import pprint
# pprint(tfidf_dict)

In [ ]:
D = np.zeros((total_doc,len(inverted_index)))
for i in tfidf_dict:
  k=tfidf_list.index(i[1])
  D[i[0]][k] = tfidf_dict[i]

#Calculating Cosine similarity

In [ ]:

def cosine_similarity1(query_dict):

 

  Q=  np.zeros((len(inverted_index)))
  res=[]
  
  for i in inverted_index.keys():
     k=tfidf_list.index(i)    
     if i in query_dict.keys():
       Q[k]=query_dict[i]

  
  for row in range(D.shape[0]):
    # print(row,"size:",row.shape)

    # print(Q,"Query Size:",Q.shape)

    # break
    #res.append(cosine_similarity(D[0],Q))
    result= 1 - spatial.distance.cosine(D[row], Q)
    res.append([row,result])
  
  return res
 
  # print(D)
  # print(Q)
  # print("Shape of D",D.shape)
  # print("shape of Q",Q.shape)


#ENTER THE QUERY


In [ ]:
query=input("Enter your query:-")
q_list= preprocess(query)

In [ ]:
query_dict={}

# q_list=query.split()
#print(q_list)
for q in q_list:
  
  if q  in inverted_index.keys():
     query_dict[q]=(inverted_index[q]['idf'])*(1+math.log2(q_list.count(q)))

  
  

   
res=cosine_similarity1(query_dict)
sorted_res= sorted(res,key=lambda x:x[1],reverse=True)
# print(res,'\n')
# print(sorted_res,'\n')

#Displaying the retrived documents

In [ ]:
pd.set_option('display.max_colwidth', None)
dataframe.iloc[[docid[0] for docid in sorted_res]].head(retried_doc)

# Relavence feedback
## Recall and Precision
## Recall = #(relevant retrieved)/#(relevant) -> tp/tp+fn

## Precision = #(relevant retrieved)/#(retrieved) -> tp/tp+fp


# print('Enter 1 if the document is relevant\n      0 if the document is irrelevant')
feedback=[]
for f in range(retried_doc):
  print('Is the document', f ,'relevant')
  x=int(input())
  feedback.append(x)


### Enter 1 if the document is relevant
### Enter 0 if the document is irrelevant

In [ ]:
print('Enter 1 if the document is relevant\n      0 if the document is irrelevant')
feedback=[]
for f in range(retried_doc):
  print('Is the document', f ,'relevant')
  x=int(input(0))
  feedback.append(x)

In [ ]:
def evaluate(feedback):
  retrieved=30
  recall=[]
  precision=[]
  relevant=feedback.count(1) 
  relavent_count=0
  for ct in range(retried_doc):
  
    if feedback[ct]==1:
      relavent_count+=1;
    recall.append(relavent_count/relevant)
    precision.append(relavent_count/(ct+1))
  print("RECALL",recall)
  print("PRECISION",precision)
  plt.xlabel("RECALL")
  plt.ylabel("PRECISION")
  plt.title("P-R Curve")
  plt.plot(recall,precision)


In [ ]:
evaluate(feedback)

#Non Trivial Tasks

Filtering movies based on release year

In [ ]:

# enter filters in the query string using /. Example:- classical singer /year 1950 
def query_with_filters(q_string):
  arr = q_string.split(' ')
  try:
    year_index = arr.index('/year')
    if arr[year_index+1].isdigit():
      input_dataframe = dataframe[dataframe['Release Year']==int(arr[year_index+1])]
      q_string = " ".join(arr[:year_index])
      arr =arr[:year_index]
    # return query,input_dataframe
    
  except ValueError:
    pass


  try:
    arr = q_string.split(' ')
    year_index = arr.index('/genre')
    if arr[year_index+1]:
      input_dataframe = dataframe[dataframe['Genre']==str(arr[year_index+1]).lower()]
      query = " ".join(arr[:year_index])
      return query,input_dataframe
  except ValueError:
    pass


  return q_string,input_dataframe

In [ ]:
print(query_with_filters('pop singer /year 1970 /genre comedy'))